In [1]:
# ダウンロードのテスト
!wget https://github.com/CPcolloquium/cp_program_book/raw/main/4_Neural-Network-Model/parameters/ctrnn-0.1-512-128-500000.pth
!wget https://github.com/CPcolloquium/cp_program_book/raw/main/4_Neural-Network-Model/src/print.py

--2024-07-05 09:15:11--  https://github.com/CPcolloquium/cp_program_book/raw/main/4_Neural-Network-Model/parameters/ctrnn-0.1-512-128-500000.pth
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/CPcolloquium/cp_program_book/main/4_Neural-Network-Model/parameters/ctrnn-0.1-512-128-500000.pth [following]
--2024-07-05 09:15:11--  https://raw.githubusercontent.com/CPcolloquium/cp_program_book/main/4_Neural-Network-Model/parameters/ctrnn-0.1-512-128-500000.pth
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6789 (6.6K) [application/octet-stream]
Saving to: ‘ctrnn-0.1-512-128-500000.pth’

ctrnn-0.1-512-

In [ ]:
import print

print.hello()